# 0. 引入依赖包
注意 cx_Oracle裤需要动态依赖

In [15]:
import pandas as pd
import networkx as nx
import numpy as np
import os
import re
from db_con.pcn_oracle_data import PCNDB
from tqdm import tqdm
os.getcwd()

'/home/wzk/Code/PCN_ROUTES'

In [2]:
os.environ['LD_LIBRARY_PATH']

':/usr/local/cuda/lib64:/home/wzk/Oracle/instantclient_19_15'

# 1. 连接数据库

In [3]:
pcnTestDB = PCNDB('PCN_TEST','784427618', '10.112.199.111:1521/LHR11G')
bus_inf = pcnTestDB.get_all_businesses()
cha_inf = pcnTestDB.get_all_channels()

# 2. 获取有效通路
数据处理

In [4]:
def get_stn_name(stn_id, pcnDB, stnDict):
    del_full_name = lambda x:x[0].split('/')[-1] if x[0] != None else None
    if stn_id not in stnDict.keys():
        stn_dict = pcnDB.get_station_inf(stn_id, tCols=['FULL_NAME'])
        stn_name = del_full_name(stn_dict['FULL_NAME']) if stn_dict['FULL_NAME'] != None else None
        stnDict[stn_id] = stn_name
    else:
        stn_name = stnDict[stn_id]
    
    return stn_name, stnDict


def ch_channel_name(channelInf, pcnDB):
    assert 'A_STATION' in channelInf.keys() and 'Z_STATION' in channelInf.keys()
    stnList_a = channelInf['A_STATION']
    stnList_z = channelInf['Z_STATION']

    stnList_a_name , stnList_z_name = [], []
    resDict = {}

    for idx, (s_a, s_z) in enumerate(zip(stnList_a, stnList_z)):
        s_a_name, resDict = get_stn_name(s_a, pcnDB, resDict)
        s_z_name, resDict = get_stn_name(s_z, pcnDB, resDict)

        stnList_a_name.append(s_a_name)
        stnList_z_name.append(s_z_name)

    channelInf['A_STATION_NAME'] = stnList_a_name
    channelInf['Z_STATION_NAME'] = stnList_z_name

    return channelInf, resDict


def del_channel_inf(channelInf, pcnDB):
    channelInf, stnDict = ch_channel_name(channelInf, pcnDB)
    stnList_a = channelInf['A_STATION_NAME']
    stnList_z = channelInf['Z_STATION_NAME']

    assert len(stnList_a) == len(stnList_z)
    idx = 0
    while idx < len(stnList_a):
        s_a = stnList_a[idx]
        s_z = stnList_z[idx]
        if s_a is None or s_z is None:
            for k in channelInf.keys():
                channelInf[k].pop(idx)
        elif s_a == s_z:
            for k in channelInf.keys():
                channelInf[k].pop(idx)
        else:
            idx += 1

    return channelInf, stnDict


def con_business_inf(b_id, pcnDB, stnDict):
    busInfDict = pcnDB.get_business_inf(b_id)
    stnID_a, stnID_z = busInfDict['A_SITE_ID'][0], busInfDict['Z_SITE_ID'][0]

    stnName_a = get_stn_name(stnID_a, pcnDB, stnDict)
    stnName_z = get_stn_name(stnID_z, pcnDB, stnDict)
    busInfDict['A_STATION_NAME'] = stnName_a
    busInfDict['Z_STATION_NAME'] = stnName_z

    return busInfDict


def con_business_stn_inf(b_id, pcnDB, stnDict):
    resDitc={}
    busInfDict = pcnDB.get_business_inf(b_id)
    stnID_a, stnID_z = busInfDict['A_SITE_ID'][0], busInfDict['Z_SITE_ID'][0]

    stnName_a,_ = get_stn_name(stnID_a, pcnDB, stnDict)
    stnName_z,_ = get_stn_name(stnID_z, pcnDB, stnDict)
    resDitc['A_STATION_NAME'] = stnName_a
    resDitc['Z_STATION_NAME'] = stnName_z

    return resDitc

 
def del_business_inf(businessInf, pcnDB, stnDict=None):
    assert 'BUSINESS_ID' in businessInf.keys() and 'CHANNEL_ID' in businessInf
    busDict = {}
    for idx, busID in enumerate(businessInf['BUSINESS_ID']):
        if busID not in busDict.keys():
            busDict[busID] = [businessInf['CHANNEL_ID'][idx]]
        else:
            busDict[busID].append(businessInf['CHANNEL_ID'][idx])

    businessInfDict ={}
    for idx, b_id in enumerate(list(busDict.keys())):
        busInfDict = con_business_stn_inf(b_id, pcnDB, stnDict)
        businessInfDict[b_id] = busInfDict

    return busDict, businessInfDict


def generate_pcnMap(channelInf):
    pcnMap = nx.Graph()
    stnList_a = channelInf['A_STATION_NAME']
    stnList_z = channelInf['Z_STATION_NAME']
    channelIdList = channelInf['OBJ_ID']

    edgeDict = {}
    for idx, (c_id, s_a, s_z) in enumerate(zip(channelIdList, stnList_a, stnList_z)):
        edgeStr = '{}-{}'.format(s_a, s_z)
        edgeStrRe = '{}-{}'.format(s_z, s_a)
        if edgeStr in edgeDict.keys():
            edgeDict[edgeStr].append(c_id)
        elif edgeStrRe in edgeDict.keys():
            edgeDict[edgeStrRe].append(c_id)
        else:
            pcnMap.add_edge(s_a, s_z)
            edgeDict[edgeStr] = [c_id]

    for e in pcnMap.edges.keys():
        edgekey = '{}-{}'.format(e[0], e[1])
        edgekeyRe = '{}-{}'.format(e[1], e[0])
        if edgekey in edgeDict.keys():
            pcnMap.edges[e]['channel_id'] = edgeDict[edgekey]
        else:
            pcnMap.edges[e]['channel_id'] = edgeDict[edgekeyRe]


    return pcnMap


def get_circuitous_bus(businessInfDict, pcnMap):
    busDelDict={}
    for idx,b_id in enumerate(businessInfDict.keys()):
        stnName_a = businessInfDict[b_id]['A_STATION_NAME']
        stnName_z = businessInfDict[b_id]['Z_STATION_NAME']

        if stnName_a not in pcnMap.nodes.keys() or stnName_z not in pcnMap.nodes.keys():
            continue
        elif stnName_a is None or stnName_z is None:
            continue
        elif stnName_a == stnName_z:
            continue
        else:
            busDelDict[b_id] = businessInfDict[b_id]

    return busDelDict

cha_inf_s, stnDict = del_channel_inf(cha_inf, pcnTestDB)
bus_inf_s,  bus_inf2_s = del_business_inf(bus_inf, pcnTestDB, stnDict)
pcnMap = generate_pcnMap(cha_inf_s)
busDelDict = get_circuitous_bus(bus_inf2_s, pcnMap)


## 2.1 获取所有可用通路
利用nx.all_simple_paths

In [5]:
def get_access_routes(busDelDict, pcnMap, cutoff=3):
    resDict={}
    routeDict = {}
    busIDList = list(busDelDict.keys())
    for idx in tqdm(range(len(busDelDict))):
        b_id = busIDList[idx]
        stn_a, stn_z = busDelDict[b_id]['A_STATION_NAME'], busDelDict[b_id]['Z_STATION_NAME']
        resKey = "{}-{}".format(stn_a, stn_z)
        resKey1 = "{}-{}".format(stn_z, stn_a)
        if resKey in resDict.keys():
            routeDict[b_id] = resDict[resKey]
        elif resKey1 in resDict.keys():
            routeDict[b_id] = resDict[resKey1]
        else:
            nx_allRoutes = nx.all_simple_paths(pcnMap,source=stn_a,target=stn_z, cutoff=cutoff)
            routes = list(nx_allRoutes)
            routeDict[b_id] = routes
            resDict[resKey] = routes

    return routeDict
    
routeDict = get_access_routes(busDelDict, pcnMap)

100%|██████████| 18083/18083 [00:21<00:00, 860.95it/s] 


# 3.路由优选

## 3.1 厂商筛选

In [49]:
def del_business_fac(chaList, pcnDB):
    assert isinstance(chaList, list)
    get_value = lambda x: x["NAME"][0] if 'NAME' in x.keys() else None

    facList = [get_value(pcnDB.get_fac_inf(c_id)) for c_id in chaList]
    facList = list(map(_del_fac_lable, facList))

    conDict = {}
    maxNum, maxName = 0, ''
    for f in facList:
        if f not in conDict.keys():
            conDict[f] = 0
        else:
            conDict[f] += 1
            if conDict[f] > maxNum:
                maxNum = conDict[f]
                maxName = f
    return maxName
    
def _del_fac_lable(facVal):
    if  not isinstance(facVal, str):
        return 'Others'
    key_words = ['华为', '依赛', '烽火', '马可尼', 'ECI北京', '中兴', '北电', '赛特']
    re_str = '|'.join(key_words)
    
    re_op_matchKeys = lambda x: [re.search(re_str, x), re.search('SDH', x)]
    re_op_delNone = lambda x: x.group(0) if x is not None else ''

    facValList = re_op_matchKeys(facVal)
    res_val = ''.join([re_op_delNone(i) for i in facValList])
    res_val = res_val if len(res_val) != 0 else facVal

    return res_val

def get_fac_inf():
    pass


In [50]:
del_business_fac(bus_inf_s['F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-33276'], pcnTestDB)

'华为SDH'

In [12]:
bus_inf_s

{'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-33273': ['0CFBF67E-4699-4808-A4A6-81FC829CF4CD-00684'],
 'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-33276': ['0CFBF67E-4699-4808-A4A6-81FC829CF4CD-00664',
  'CD996599-B045-43B0-9785-F2B51C807217-43869',
  '0E0BBF9D-2FA1-40AF-949D-7D046035E6DF-20996'],
 '648C6BD8-20AE-4B1B-88C6-F5064C708F28-77700': ['4024CCE8-667E-41B7-8C1C-4D7194D19C8F-29460'],
 'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-33280': ['0CFBF67E-4699-4808-A4A6-81FC829CF4CD-00644'],
 'B4409567-89B8-4B77-B1CC-F2BEC964941C-00103': ['7A68CAD5-35D3-4C4D-9C7F-387E25F709FE-06707'],
 '6CACACE9-D237-4240-B390-216E10B6DBB5-22770': ['C24667E1-0A58-4818-B446-8B40E305A74D-59023'],
 '648C6BD8-20AE-4B1B-88C6-F5064C708F28-77614': ['4024CCE8-667E-41B7-8C1C-4D7194D19C8F-29570'],
 'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-33207': ['0CFBF67E-4699-4808-A4A6-81FC829CF4CD-63555'],
 'F4EC8E82-42BF-484A-922B-68F685720FDF-00001': ['7672D6BC-90F8-4B94-B2BD-49AF0858A2B0-53369'],
 'B4409567-89B8-4B77-B1CC-F2BEC964941C-00357': ['

# -1.测试

In [10]:
busDelDict

{'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-33276': {'A_STATION_NAME': '龙岗电厂',
  'Z_STATION_NAME': '500kV花都变'},
 '648C6BD8-20AE-4B1B-88C6-F5064C708F28-77700': {'A_STATION_NAME': '220kV溯源变',
  'Z_STATION_NAME': '35kV赵村变'},
 'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-33280': {'A_STATION_NAME': '220kV薛坡变',
  'Z_STATION_NAME': '500kV花都变'},
 'B4409567-89B8-4B77-B1CC-F2BEC964941C-00103': {'A_STATION_NAME': '500kV花都变',
  'Z_STATION_NAME': '220kV薛坡变'},
 '6CACACE9-D237-4240-B390-216E10B6DBB5-22770': {'A_STATION_NAME': '河南省电力公司西区',
  'Z_STATION_NAME': '漯河市供电公司'},
 '648C6BD8-20AE-4B1B-88C6-F5064C708F28-77614': {'A_STATION_NAME': '220kV兴泰变',
  'Z_STATION_NAME': '35kV赵村变'},
 'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-33207': {'A_STATION_NAME': '220kV孟平变',
  'Z_STATION_NAME': '商丘市供电公司'},
 'F4EC8E82-42BF-484A-922B-68F685720FDF-00001': {'A_STATION_NAME': '220kV宛北变',
  'Z_STATION_NAME': '220kV邓州变'},
 'B4409567-89B8-4B77-B1CC-F2BEC964941C-00357': {'A_STATION_NAME': '500kV湛河变',
  'Z_STATION_NAME': '220kV乔庄变'},
 'B44095

In [8]:
print("{:.2f}".format(np.array([d for _, d in pcnMap.degree]).mean()))


7.82


In [9]:
print(" MAP INF ".center(50, "*"))
print('\t* Nodes num:{}'.format(len(pcnMap.nodes)))
print('\t* Edges num:{}'.format(len(pcnMap.edges)))
print(" MAP INF ".center(50,'*'))

******************** MAP INF *********************
	* Nodes num:3048
	* Edges num:11911
******************** MAP INF *********************


In [10]:
c, v = pcnTestDB._exec_sql("SELECT * FROM T_SPC_SITE WHERE OBJ_ID = 'F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00391'")

In [11]:
bus_inf_s['94FAF1DB-56F7-4C1E-B539-A542C1474E37-29055']

['F8165186-C245-4465-8A2A-DB5C9ABE134D-44585']

In [12]:
bus_inf_s.keys().__len__()

22173

In [13]:
pcnTestDB.get_channel_inf(c_id='F8165186-C245-4465-8A2A-DB5C9ABE134D-44585')

{'OBJ_ID': ['F8165186-C245-4465-8A2A-DB5C9ABE134D-44585'],
 'NAME': ['E_2M[220kV越翎变:1框-2槽-调度专网（110kv奥封风电场-获嘉变）端口~时隙:1<>500kV获嘉变OptiX OSN 7500:1框-2槽-CPOS8/2/0端口~时隙:1-1-4-2]'],
 'CHANNEL_TYPE': ['1'],
 'RATE': ['5'],
 'A_RES_ID': ['2392F5E9-C562-4FEC-A860-D43216C74BF6-24762'],
 'Z_RES_ID': ['7BC33DA9-B603-41D6-A005-8C9651C56C4A-75842'],
 'A_STATION': ['085E98B2-66D4-4340-A21F-DC531C5F3A3D-76049'],
 'Z_STATION': ['F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00152']}

In [14]:
'奥封风电场' in cha_inf_s['Z_STATION']

False

In [16]:
pcnTestDB.get_business_inf(b_id='94FAF1DB-56F7-4C1E-B539-A542C1474E37-29055')

{'OBJ_ID': ['94FAF1DB-56F7-4C1E-B539-A542C1474E37-29055'],
 'FULL_NAME': ['豫/省公司自动化业务系统/(奥封风电场～500kV嘉和变) 调度自动化业务01'],
 'BUZ_TYPE': ['3'],
 'A_SITE_ID': ['4FF9B58D-A99F-46F8-82E2-9745D1B555D8-79715'],
 'Z_SITE_ID': ['F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-00255'],
 'DISPATCH_LEVEL': ['3'],
 'BUZ_RATE': ['2M']}

In [18]:
pcnDBInf={
        'USER': 'PCN_TEST',
        'PASS': '784427618',
        'SID': '172.17.0.2:1521/LHR11G'
    } 
print(" PCN DataBase INF ".center(50, '*'))
for k in pcnDBInf.keys():
    if k != 'PASS':
        print("\t* {}: {}".format(k, pcnDBInf[k]))
print(" Successfully connected! ".center(50, '*'))


**************** PCN DataBase INF ****************
	* USER: PCN_TEST
	* SID: 172.17.0.2:1521/LHR11G
************ Successfully connected! *************
